Configuring PyTorch to use GPU

In [3]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

Mounted at /content/drive


In [4]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'MyDrive/Github/' 
# replace with your Github username 
GIT_USERNAME = "YOUR_USERNAME" 
# definitely replace with your
GIT_TOKEN = "YOUR_TOKEN"  
# Replace with your github repository in this case we want 
GIT_REPOSITORY = "SemEval-2022" 

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
print("PROJECT_PATH: ", PROJECT_PATH)   
!mkdir "{PROJECT_PATH}"    
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

In [ ]:
%cd "{PROJECT_PATH}"
!git clone "{GIT_PATH}"

In [11]:
!pip install transformers=='2.8.0'

     |████████████████████████████████| 563 kB 7.9 MB/s 
     |████████████████████████████████| 5.6 MB 20.2 MB/s 
     |████████████████████████████████| 895 kB 52.7 MB/s 
     |████████████████████████████████| 131 kB 51.8 MB/s 
     |████████████████████████████████| 1.2 MB 24.9 MB/s 
     |████████████████████████████████| 8.4 MB 54.1 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 138 kB 54.7 MB/s 
     |████████████████████████████████| 127 kB 49.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
import sys  
sys.path.insert(0, '/content/drive/MyDrive/Github/SemEval-2022/pcl')
%cd /content/drive/MyDrive/Github/SemEval-2022/pcl

/content/drive/MyDrive/Github/SemEval-2022/pcl


In [8]:
from dont_patronize_me import DontPatronizeMe

# Initialize a dpm (Don't Patronize Me) object.
# It takes two areguments as input: 
# (1) Path to the directory containing the training set files, which is the root directory of this notebook.
# (2) Path to the test set, which will be released when the evaluation phase begins. In this example, 
# we use the dataset for Subtask 1, which the code will load without labels.
dpm = DontPatronizeMe('/content/drive/MyDrive/Github/SemEval-2022/dataset/pcl', '.dontpatronizeme_pcl.tsv')

## Load Subtask 1 Data

In [9]:
# This method loads the subtask 1 data
dpm.load_task1()
# which we can then access as a dataframe
dpm.train_task1_df.head()

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"we 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"in libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""white house press secretary sean spicer said ...",0,0
3,4,@@7811231,disabled,nz,council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" just like we received migrants fleeing el ...",0,0


In [27]:
import pandas as pd
import re
import os
import math
import torch
from torch.nn import BCEWithLogitsLoss, NLLLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, TFXLNetModel, XLNetLMHeadModel, XLNetConfig, XLNetForSequenceClassification
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm, trange

X = dpm.train_task1_df['text']
y = dpm.train_task1_df['label']

## Preprocessing Text

In [49]:
from nltk.tokenize import WordPunctTokenizer
import re
from bs4 import BeautifulSoup
import itertools

tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'

def text_cleaner(text):
    # removing UTF-8 BOM (Byte Order Mark)
    try:
        text = text.decode("utf-8-sig").replace(u"\ufffd", "?") # The UTF-8 BOM is a sequence of bytes (EF BB BF) that allows the reader to identify a file as being encoded in UTF-8
    except:
        text = text
    
    
    #replace consecutive non-ASCII characters with a space
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    
    # HTML encoding
    soup = BeautifulSoup(text, 'lxml') #HTML encoding has not been converted to text, and ended up in text field as ‘&amp’,’&quot’,etc.
    text = soup.get_text()
    
    # Removing URLs
    text = re.sub(pat2, '', text)
    
    # Removing punctuations
    text = re.sub("[\.\,\!\?\:\;\-\=\(\)\[\]\"\%\*\#\@]", " ", text)
    text = re.sub("\s\'\s", " ", text)
    
    # Expanding/Collapsing punctuated words
    text = re.sub("\s\'", "\'", text)
    text = re.sub("n\'t", "not", text)

    # Remove extra whitespaces
    text = re.sub(' +', ' ', text)

    # Fix misspelled words
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))# checking that each character should occur not more than 2 times in every word

    # Remove leading and trailing whitespaces
    text = text.strip()

    # Change to lowercase

    text = text.lower()
    
    return text

In [53]:
X = list(map(text_cleaner, X))

skin needling is a safe and precise way to traumatise the skin to stimulate collagen , based on the above principal . it results in a smoother , firmer and younger looking skin .
skin needling is a safe and precise way to traumatise the skin to stimulate collagen based on the above principal it results in a smoother firmer and younger looking skin


To feed our text to XLNet, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary.

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased", do_lower_case=True)

## Tokenization

XLNet requires specifically formatted inputs. For each tokenized input sentence, we need to create:

    input ids: a sequence of integers identifying each input token to its index number in the XLNet tokenizer vocabulary
    segment mask: (optional) a sequence of 1s and 0s used to identify whether the input is one sentence or two sentences long. For one sentence inputs, this is simply a sequence of 0s. For two sentence inputs, there is a 0 for each token of the first sentence, followed by a 1 for each token of the second sentence
    attention mask: (optional) a sequence of 1s and 0s, with 1s for all input tokens and 0s for all padding tokens
    labels: a single value of 1 or 0. In our task 1 means “PCL” and 0 means “No PCL”

Although we can have variable length input sentences, XLNet does requires our input arrays to be the same size. We address this by first choosing a maximum sentence length, and then padding and truncating our inputs until every input sequence is of the same length.

To “pad” our inputs in this context means that if a sentence is shorter than the maximum sentence length, we simply add 0s to the end of the sequence until it is the maximum sentence length.

If a sentence is longer than the maximum sentence length, then we simply truncate the end of the sequence, discarding anything that does not fit into our maximum sentence length.

We pad and truncate our sequences so that they all become of length maxlen (“post” indicates that we want to pad and truncate at the end of the sequence, as opposed to the beginning) pad_sequences is a utility function that we’re borrowing from Keras. It simply handles the truncating and padding of Python lists.


In [ ]:
def tokenize_inputs(text_list, tokenizer, num_embeddings=120):
    """
    Tokenizes the input text input into ids. Appends the appropriate special
    characters to the end of the text to denote end of sentence. Truncate or pad
    the appropriate sequence length.
    """
    # tokenize the text, then truncate sequence to the desired length minus 2 for
    # the 2 special characters
    tokenized_texts = list(map(lambda t: tokenizer.tokenize(t)[:num_embeddings-2], text_list))
    # convert tokenized text into numeric ids for the appropriate LM
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    # append special token "<s>" and </s> to end of sentence
    input_ids = [tokenizer.build_inputs_with_special_tokens(x) for x in input_ids]
    # pad sequences
    input_ids = pad_sequences(input_ids, maxlen=num_embeddings, dtype="long", truncating="post", padding="post")
    return input_ids

def create_attn_masks(input_ids):
    """
    Create attention masks to tell model whether attention should be applied to
    the input id tokens. Do not want to perform attention on padding tokens.
    """
    # Create attention masks
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return attention_masks

In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs.

input_ids = tokenize_inputs(X, tokenizer, num_embeddings=120)
attention_masks = create_attn_masks(input_ids)

# Convert the lists into tensors.
input_ids = torch.from_numpy(input_ids)
attention_masks = torch.tensor(attention_masks)
y = torch.tensor(y)

# Print sentence 0, now as a list of IDs.
print('Original: ', X[1])
print('Token IDs:', input_ids[1])
print('Attention Masks:', attention_masks[1])

## Generating Data Splits

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, y)

# Create a 75-15-10 train-validation-test split.

# Calculate the number of samples to include in each set.
train_size = int(0.75 * len(dataset))
val_size = round(0.6*(len(dataset) - train_size))
test_size = len(dataset) - train_size - val_size

print(len(dataset))
print(train_size)
print(val_size)
print(test_size)

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


We'll also create an iterator for our dataset using the torch DataLoader class. This helps save on memory during training because, unlike a for loop, with an iterator the entire dataset does not need to be loaded into memory.

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. Batch size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

# Load pretrained XLNet model

In [ ]:
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 2)
model = model.to(device)

# Setting up Hyperparameters


In [ ]:
EPOCHS = 3
BATCH_SIZE = 32

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)